# Pandas

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from IPython.display import Image, IFrame

**pandas** is a Python package providing fast, flexible, and expressive data structures designed to work with *relational* or *labeled* data. It is a fundamental high-level building block for doing practical, real world data analysis in Python. 

pandas is well suited for:

- Tabular data with heterogeneously-typed columns, as in an SQL table or Excel spreadsheet
- Ordered and unordered (not necessarily fixed-frequency) time series data.
- Arbitrary matrix data (homogeneously typed or heterogeneous) with row and column labels
- Any other form of observational / statistical data sets. The data actually need not be labeled at all to be placed into a pandas data structure


Key features:  
    
- Easy handling of **missing data**
- **Size mutability**: columns can be inserted and deleted from DataFrame and higher dimensional objects
- Automatic and explicit **data alignment**: objects can be explicitly aligned to a set of labels, or the data can be aligned automatically
- Powerful, flexible **group by functionality** to perform split-apply-combine operations on data sets
- Intelligent label-based **slicing, fancy indexing, and subsetting** of large data sets
- Intuitive **merging and joining** data sets
- Flexible **reshaping and pivoting** of data sets
- **Hierarchical labeling** of axes
- Robust **IO tools** for loading data from flat files, Excel files, databases, and HDF5
- **Time series functionality**: date range generation and frequency conversion, moving window statistics, moving window linear regressions, date shifting and lagging, etc.

In [ ]:
IFrame(src='http://pandas.pydata.org/index.html', width=1000, height=350)

### The conventional way to import pandas is: 

```python
import pandas as pd
```

In [ ]:
import pandas as pd

In [ ]:
print(pd.__version__)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

some options setting 

In [ ]:
pd.set_option("display.width", 80)
# toggle the line below that if one doesnt want DataFrames displayed as HTML tables
#pd.set_option("notebook_repr_html", False) 
pd.set_option("notebook_repr_html", True) 

### Pandas's data structures and functionalities will be familiar to R users, there's a section on Pandas's website where Wes McKinney gives some translation of common idioms / operations between R and Pandas 

In [ ]:
IFrame('https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_r.html', width=1000, height=350)

## Pandas data structures

### Series


A **Series** is a single vector of data values (think a NumPy array with shape N or (N,1)) with an **index** that labels each element in the vector.

#### Series constructions

In [ ]:
np.random.seed(42)

In [ ]:
x = np.random.normal(0,1,(10,))

In [ ]:
x.shape

In [ ]:
type(x)

In [ ]:
plt.plot(x)

In [ ]:
a = pd.Series(x)

In [ ]:
a

In [ ]:
a.index

In [ ]:
a.values

#### You can define your own **index**

In [ ]:
a = pd.Series(x, index=np.arange(1,11))

In [ ]:
a

In [ ]:
a.index

In [ ]:
a.plot()

In [ ]:
a = pd.Series(np.random.normal(0,1,(5,)), index=['a','b','c','d','e'], name='my series')

In [ ]:
a.index

In [ ]:
a.plot()

Pandas objects expose some powerful, **high level plotting functions** (built on top of Matplotlib)

In [ ]:
f, ax = plt.subplots()
a.plot(kind='bar', rot=0, color='r', width=0.9, ax=ax)
ax.set_title('BLA')

same plot in matplotlib

In [ ]:
type(a)

In [ ]:
f, ax = plt.subplots()
bars = ax.bar(np.arange(len(a)), a.values, color='r', align='center', width=0.9)
ax.set_xticks(np.arange(len(a)))
ax.set_xlim(-0.5, len(a)-0.5)
ax.set_xticklabels(a.index)
ax.set_title(a.name)

#### Series indexing

Selecting from a Series is easy, using the corresponding index key (like a dict)

In [ ]:
a

In [ ]:
a['c']

slices are permitted 

In [ ]:
a['a':'c'] ### Note the difference with standard Python / Numpy positional, integer indexing

In [ ]:
a['c':]

deleting an element 

In [ ]:
a

In [ ]:
a.drop('d')

In [ ]:
a

In [ ]:
a.drop('a', inplace=True)

In [ ]:
a

Adding an element is (to my knowledge) not straightforward

In [ ]:
a.append(pd.Series({'f':5}))

In [ ]:
a

In [ ]:
a = a.sort_index(ascending=False)

In [ ]:
a

Mathematical operations involving two series will perform operations by *aligning indices*.

1. The union of all indices is created
2. The mathematical operation is performed on matching indices. 

Indices that do not match are given the value NaN (not a number), and values are computed for all unique pairs of repeated indices.

In [ ]:
s1 = pd.Series(np.arange(1.0,4.0),index=['a','b','c'])
s2 = pd.Series(np.arange(1.0,5.0),index=['b','c','d','e'])

In [ ]:
s2

In [ ]:
s3 = s1 + s2

In [ ]:
s3

NaNs are ignored in all operations 

In [ ]:
s3.mean()

You can drop them from the Series

In [ ]:
s3

In [ ]:
s4 = s3.dropna()

In [ ]:
s4

Or use the `fillna` method to replace them by a value

In [ ]:
s3.fillna(-999)

In [ ]:
s5 = s3.fillna(s3.mean())

In [ ]:
s5

In [ ]:
pd.date_range(start='1998-1-1', end='1998-12-31')

In [ ]:
ts = pd.Series(np.arange(365), index=pd.date_range(start='1998-1-1', end='1998-12-31'))

In [ ]:
ts.head()

In [ ]:
ts.rolling(30, center=True, min_periods=30).mean()

### DataFrames

**DataFrames** are IMHO one of the most powerful data structures in the Python / data analysis world. 

They can be viewed as a *collection* of named Series. They feature two **indexes**, respectively for the rows and the columns, and can contain heteregoneous data types (although it must be consistent *within* each column). 
Note that a DataFrame index, either along the rows or the columns (or both !) can contain more than one level, they are called **hierarchical indexes** and allows the representation of complex data organisation. 

If the index along the rows of a DataFrame is of **datetime** type, all the methods exposed for the Series (re-sampling, shifting, truncating, etc) are available for the DataFrame.

#### DataFrame constructions

In [ ]:
import string # part of the standard library
idx = list(string.ascii_lowercase[0:10])
print(idx)

In [ ]:
df = pd.DataFrame(np.arange(100).reshape(10,10),\
                  columns=idx,index=np.arange(1,11))

In [ ]:
df

In [ ]:
### here I am creating a DataFrame from a dictionnary

df = pd.DataFrame({'a' : np.random.random(5),\
                   'b' : np.random.random(5),\
                   'c': np.random.random(5)}, index=np.arange(1,6))

In [ ]:
df

#### Indexing a DataFrame

different ways, return either a Series or a DataFrame

In [ ]:
df

In [ ]:
df['a']

In [ ]:
type(df[['a']]) 

In [ ]:
df[['a']]

In [ ]:
type(df['a'])

To access a particular *row* instead of a column, you use the *loc* method

In [ ]:
df

In [ ]:
df.loc[[3],'a':'b']

access values per **positional indices** only

In [ ]:
df.iloc[2,0:2]

In [ ]:
df.loc[3,:]['a':'b']

In [ ]:
df.loc[3:5,:][['a','c']]

you can also use the `loc` method, giving it both row **AND** columns indexes 
(the indexing is based on the *label*, **NOT** on the *position*)

In [ ]:
df

In [ ]:
df.loc[[3,5],:]

conditional indexing or subsetting of a DataFrame 

In [ ]:
df

In [ ]:
subset = df[df['a'] <= 0.4]

In [ ]:
subset

#### queries

as from Pandas version 0.14, you can build complex *database-like* queries on DataFrames

In [ ]:
df

In [ ]:
df.query('a > b')

In [ ]:
df.query('(a > b) & (b < c)')

#### Extending a DataFrame

Adding a column is easy 

In [ ]:
df

In [ ]:
df['d'] = np.random.random(5)

In [ ]:
df

### or 

In [ ]:
df.loc[:,'d'] = np.random.random(5)

In [ ]:
df

The following works because Pandas understands that a single value must be repeated over the row length

In [ ]:
df.loc[:,'e'] = 5

In [ ]:
df

In [ ]:
df2 = pd.DataFrame(df.values, index=range(6,11), columns=['a', 'b','c','d','e'])

In [ ]:
df2

In [ ]:
df

In [ ]:
df.rename({'e':'bla'}, axis=1)

The following doesn't work because there's no way to tell **where** to insert the missing value (align to 1st or last index ?)

In [ ]:
df

In [ ]:
df.loc[:,'f'] = np.random.random(4)

Unless we make a series out of it, with a index matching at least partly the DataFrame (row) index

In [ ]:
df.loc[:,'f'] = pd.Series(np.random.random(4), index=np.arange(1,5)) #

In [ ]:
df

#### Useful DataFrames methods

##### applying a numpy function

In [ ]:
df.apply(np.sqrt) # or np.sqrt(df)

##### summarizing data

##### replacing values

In [ ]:
df.head()

In [ ]:
df['e'].replace(5, 0.0, inplace=True) 
# you can use dictionnary to multiple replacements

In [ ]:
df

In [ ]:
df.replace({'e':{0.0:6}})

##### assign 

In [ ]:
df.assign(f = df['a'] + df['e'])

that allows you to write very succint code

In [ ]:
df2 = df.query('a > 0.4').assign(f = df['a'] + df['e'])

In [ ]:
df2

In [ ]:
df

#### DataFrame high-level plotting functions

In [ ]:
df.plot()

In [ ]:
df.plot(subplots=True, figsize=(8,10));

In [ ]:
df.plot(figsize=(8,12), \
                          subplots=True, \
                          sharex=True, \
                          kind='bar', rot=0); 

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
 
    
iris = load_iris()
 
iris = pd.DataFrame(iris.data, columns=iris.feature_names)

In [ ]:
iris.head()

In [ ]:
iris.hist(); 

In [ ]:
from pandas.plotting import scatter_matrix

In [ ]:
scatter_matrix(iris, alpha=0.4, figsize=(10, 10), diagonal='kde'); 

In [ ]:
IFrame('http://pandas.pydata.org/pandas-docs/version/1.0.2/user_guide/visualization.html', width=1200, height=350)

### Input and Output in pandas

Pandas has **very powerful IO methods**, allowing to load csv, excel, tab-delimited files very easily. Pandas DataFrames can also be 
saved also in csv, excel files. 

Other supported file types are: 

* JSON (JavaScript Object Notation)
* HDF5 
* HTML (to e.g. read tables contained in HTML documents)
* SQL (Pandas can **query directly from SQL databases** thanks to [SQLAlchemy](http://www.sqlalchemy.org/))
* ...

#### reading from CSV or Excel files

In [ ]:
SOI = pd.read_csv('../data/NIWA_SOI_1941_2010.csv') 

In [ ]:
SOI.head()

In [ ]:
SOI.tail()

In [ ]:
SOI = pd.read_csv('../data/NIWA_SOI_1941_2010.csv', index_col=0)

In [ ]:
SOI.head()

In [ ]:
SOI = pd.read_csv('../data/NIWA_SOI_1941_2010.csv', index_col='Year')

In [ ]:
SOI.head()

In [ ]:
SOI.loc[1950:,:]

#### Saving in csv, excel etc

In [ ]:
recent_SOI = SOI.loc[2013:,:]

In [ ]:
recent_SOI.head()

##### saving to a csv file

In [ ]:
recent_SOI.to_csv('../data/recent_SOI.csv')

##### saving to an Excel file

In [ ]:
recent_SOI.to_excel('../data/recent_SOI.xlsx', header=False, sheet_name='SOI')

##### if you want to save each year in a different sheet in an excel spreadsheet 

In [ ]:
writer = pd.ExcelWriter('../data/recent_SOI_sheets.xlsx', engine='xlsxwriter')

In [ ]:
for year in range(2013, 2017+1): 
    year_soi = SOI.loc[year,:]
    year_soi.to_excel(writer, sheet_name=str(year))

In [ ]:
writer.save()

#### dealing with separators in text files

In [ ]:
url = 'http://www.jamstec.go.jp/frsgc/research/d1/iod/DATA/emi.weekly.txt'

In [ ]:
IFrame(url, width=900, height=200)

In [ ]:
EMI_weekly = pd.read_table(url)

In [ ]:
EMI_weekly.head()

In [ ]:
EMI_weekly = pd.read_table(url, header=None)

In [ ]:
EMI_weekly.head()

#### You can use [regular expressions](http://en.wikipedia.org/wiki/Regular_expression) to specify what delimiters to use. 

To know more about regular expressions and their use from Python via the [re](https://docs.python.org/2/library/re.html) package, read: [http://www.ucs.cam.ac.uk/docs/course-notes/unix-courses/PythonRE/files/PythonRE.pdf](http://www.ucs.cam.ac.uk/docs/course-notes/unix-courses/PythonRE/files/PythonRE.pdf)

In [ ]:
EMI_weekly = pd.read_table(url, \
                            header=None, sep=r'[:, \s*]', parse_dates={'date':[0,1,2]}, engine='python',\
                            index_col='date', usecols=[0,1,2,4])

In [ ]:
EMI_weekly.head()

### or you can parse the dates after loading the spaces separated columns, then set them as the index

In [ ]:
EMI_weekly = pd.read_table(url, sep=r'\s+', engine='python', header=None)

In [ ]:
EMI_weekly.head()

In [ ]:
EMI_weekly.index = pd.to_datetime(EMI_weekly.loc[:,0], format="%Y:%m:%d:0")

In [ ]:
EMI_weekly.index

In [ ]:
EMI_weekly = EMI_weekly.drop(0, axis=1)

In [ ]:
EMI_weekly.columns = ['EMI']

In [ ]:
EMI_weekly.index.name = 'date'

In [ ]:
EMI_weekly.head()

In [ ]:
EMI_weekly.plot();

### groupby operations in Pandas 

The **groupby** method is a very powerful method of pandas DataFrames, in a nutschell it allows you to

1. **split** your data according to unique values of a variable (or unique *combinations* of *N* variables)

2. **apply** some operation to the groups thus defined, either an *aggregation* or *transformation* method 

3. **combine** the results into a DataFrame

This process is illustrated below, where the operation is here calculating the mean of the groups's values

A very nice explanation of the **groupby** method, with examples, is available from Pandas's documentation at:  

[http://pandas.pydata.org/pandas-docs/stable/groupby.html](http://pandas.pydata.org/pandas-docs/stable/groupby.html)

and a short tutorial on Wes McKinney's blog [here](http://wesmckinney.com/blog/?p=125)

In [ ]:
Image(filename='images/split-apply-combine.png', width=800)

In [ ]:
url = "https://www.cpc.ncep.noaa.gov/data/indices/ersst5.nino.mth.81-10.ascii"

In [ ]:
from shutil import which 

In [ ]:
which('wget')

In [ ]:
from subprocess import call

In [ ]:
cmd = f"{which('wget')} -P ../data https://www.cpc.ncep.noaa.gov/data/indices/ersst5.nino.mth.81-10.ascii"

In [ ]:
r = call(cmd, shell=True)

In [ ]:
r

In [ ]:
!ls ../data/ersst*

In [ ]:
# if the network confirguration allows it, you can read directly off the URL (ftp):

data = pd.read_table(url, sep='\s+')

In [ ]:
data.tail()

I only keep the raw - monthly - values of NINO 3.4 

In [ ]:
nino = data.loc[:,['YR','MON','NINO3.4']]

In [ ]:
nino.head()

Now I want to calculate a climatology (over the whole period available)

I first group by UNIQUE values of the variable `MON` (the month), I should get 12 groups

In [ ]:
groups = nino.groupby('MON')

In [ ]:
for month, group in groups:
    print(month)
    print(group.head())

In [ ]:
climatology = groups.mean()

Same as 

    climatology = groups.aggregate(np.mean)
    
    

In [ ]:
climatology = nino.groupby('MON').mean()

In [ ]:
climatology

In [ ]:
climatology['NINO3.4']

In [ ]:
type(climatology)

In [ ]:
f, ax = plt.subplots()

climatology.loc[:,'NINO3.4'].plot(ax=ax, kind='bar',ylim=[26,28], rot=0, width=0.9)

ax.set_xticklabels(list('JFMAMJJASOND'));

In [ ]:
nino.groupby('MON')[['NINO3.4']].mean()

Now suppose we want to apply a function that doesnt **aggregate** the values in the groups (such as *sum*, or *mean*) but rather want to **apply a function** to those values ... 

An example would be calculating the standardized anomalies per month (to each value subtract the mean of the corresponding month, then divide by the standard-deviation)

In [ ]:
def zscore(x): 
    z = (x - x.mean()) / x.std()
    return z

In [ ]:
nino.head()

In [ ]:
transformed = nino.groupby('MON').apply(zscore)

In [ ]:
transformed.head()

In [ ]:
transformed.loc[:,['NINO3.4']].plot()

In [ ]:
nino.loc[:,'NINO3.4'].plot()

In [ ]:
transformed.loc[:,'NINO3.4'].plot()

Now we want calculate (just) the anomalies WRT to the climatology, but with a **normal established over 1981 - 2010**

In [ ]:
nino.head()

### remember that we can make use of the *query* method of Pandas DataFrames to select the climatological period

In [ ]:
nino.query('YR >= 1981 & YR <= 2010').head()

### now defines the new function 

In [ ]:
def demean(x): 
    z = x - x.query('YR >= 1981 & YR <= 2010').mean()
    return z

In [ ]:
anoms = nino.groupby('MON').apply(demean)

In [ ]:
anoms.loc[:,['NINO3.4']].plot()

In [ ]:
data.loc[:,'ANOM.3'].plot() # Should be the same

In [ ]:
anoms.loc[:,'NINO3.4'].plot()

In [ ]:
f, ax = plt.subplots(figsize=(12,7))
(anoms.loc[:,'NINO3.4'] + 1).plot(ax=ax)
data.loc[:,'ANOM.3'].plot(ax=ax)

#### Exercise

The file **Daily_clim_data_Auckland.csv** in the `data` directory (`./data/Daily_clim_data_Auckland.csv`) contains 
daily values for rainfall, minimum and maximum temperatures for Auckland from 1972 to now.  

1. Read the data in a Pandas DataFrame, correctly setting up the index as a datetime index 
2. calculate monthly means for the temperature data, monthly totals for rainfall 
3. calculate and plot a climatology over the period 1981 - 2000 for all parameters 
4. calculate the anomalies WRT this climatology for all parameters 
5. plots the time-series of minimum and maximum temperature 
6. dig into the Scipy documentation to find how to calculate the linear trend over the period for the temperature data 
7. calculate and plot the trend 

### Series with a date / datetime index (timeseries)

Series can have indexes representing dates / times 

In [ ]:
a

In [ ]:
a.index = pd.date_range(start='2014-1-1', periods=len(a)) # default 'period' is daily

In [ ]:
a

Below are the codes for the frequencies that Pandas supports, note that you can specify an integer multiple of a frequency: i.e. `2D` means 2 days 

<div style='float: center'>
  <img style='width: 400px' src="./images/pandas_freqs.png"></img>
</div>

In [ ]:
a.head()

In [ ]:
a.index

In [ ]:
### but you can convert it to an numpy array of python datetime objects if you want
py_datetimes = a.index.to_pydatetime()

In [ ]:
py_datetimes

And a number of useful methods for manipulation of time series is exposed

In [ ]:
a

In [ ]:
a.tshift(-1)

In [ ]:
### the ```shift``` method makes it easy e.g. to compare series with lead / lags 
a.shift(periods=-1)

In [ ]:
a

In [ ]:
### and the ```truncate`` method allows easy selection of time-slices
a.truncate(before='2014-1-2', after='2014-1-4')

#### correctly parsing date(times) when date and time information are contained in different columns

In [ ]:
data = pd.read_excel('../data/ISO_datetime.xls', sheet_name='Sheet1')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data = pd.read_excel('../data/ISO_datetime.xls', sheet_name='Sheet1', index_col=0, parse_dates=True)

In [ ]:
data.head()

In [ ]:
data.index

when the year, month and day are contained in different columns

In [ ]:
ymd = pd.read_excel('../data/year_month_day.xlsx', sheet_name='Sheet1')

In [ ]:
ymd.head()

In [ ]:
pd.to_datetime(ymd.loc[:,['year','month','day']])

In [ ]:
ymd.index = pd.to_datetime(ymd.loc[:,['year','month','day']])

In [ ]:
ymd

In [ ]:
myd = pd.read_excel('../data/year_month_day.xlsx', sheet_name='Sheet2'); myd.head()

In [ ]:
pd.to_datetime(myd.loc[:,['year','month','day']])

In [ ]:
myd.index = pd.to_datetime(myd.loc[:,['year','month','day']])

another way to do it, might come handy when column names are NOT `year`,`month`,`day`

In [ ]:
from datetime import datetime

In [ ]:
myd.index = myd[['year', 'month', 'day']].apply(lambda d: datetime(*d), axis = 1)

In [ ]:
myd.head()

#### dealing with missing values -- and missing dates -- in Pandas

read some daily data from CLIFLO (thanks Petra)

In [ ]:
# !open ./data/Wellington_rain.xlsx

The Excel file has a header and a footer, we need to get rid of the first 8 and last 8 lines 

In [ ]:
data = pd.read_excel('../data/Wellington_rain.xlsx', skiprows=8, skipfooter=8)

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.index = data.loc[:,'Date(NZST)']

In [ ]:
data = data.loc[:,['Amount(mm)']]

In [ ]:
data.head()

In [ ]:
data = data.sort_index()

In [ ]:
data.plot()

In [ ]:
data.index[0]

In [ ]:
data.index[-1]

In [ ]:
len(data)

In [ ]:
dates = pd.date_range(start=data.index[0], end=data.index[-1], freq='D')

In [ ]:
len(dates)

In [ ]:
data_reindexed = data.reindex(dates)

In [ ]:
data_reindexed.plot()

In [ ]:
data_reindexed = data_reindexed.loc['2014':,:]

In [ ]:
data_reindexed.plot()

In [ ]:
any(pd.isnull(data_reindexed))

In [ ]:
data_reindexed.interpolate().plot()

#### Another example: UP-sampling and linear interpolation 

MSLP at Ouloup station (Ouvea): 1 hour interval, but need to be on 15 minutes interval, with linear interpolation

In [ ]:
data = pd.read_csv('../data/OULOUP_PMER.csv', index_col=0, parse_dates=True)

In [ ]:
data.head()

In [ ]:
data = data.loc[:,['PMER']]

In [ ]:
data.head()

In [ ]:
data.plot()

In [ ]:
data.head()

In [ ]:
data.resample?

In [ ]:
data.head()

In [ ]:
data.resample('1D').mean()

In [ ]:
upsampled = data.resample('15min')

In [ ]:
upsampled

In [ ]:
upsampled = upsampled.interpolate()

In [ ]:
upsampled.head()

In [ ]:
data.plot()

In [ ]:
upsampled.plot()

#### downsampling: going from higher to lower frequency

In [ ]:
data.head()

In [ ]:
downsampled = data.resample('1D').mean()

In [ ]:
downsampled.plot()

#### rolling window operations 

In [ ]:
downsampled

In [ ]:
rolling = downsampled.rolling(3, min_periods=3)

In [ ]:
rolling.mean().plot()

In [ ]:
rmean3days = downsampled.rolling(3, min_periods=3).mean()

In [ ]:
rmean3days.plot()

In [ ]:
f, ax = plt.subplots()
downsampled.rolling(3, min_periods=3).min().plot(ax=ax)
downsampled.rolling(3, min_periods=3).max().plot(ax=ax)

#### you can define your OWN function and pass it to the rolling dataframe object, using the `apply` method

In [ ]:
def maxmin(x):
    return x.max() - x.min()

In [ ]:
maxmin_data = downsampled.rolling(3, min_periods=3).apply(maxmin)

In [ ]:
maxmin_data.plot()

In [ ]:
downsampled5D_maxmin = downsampled.resample('5D').apply(maxmin)

In [ ]:
downsampled5D_maxmin.plot()